In [1]:
# Using suds SOAP client to connect to USGS Elevation Service
from suds.client import Client
USGS = Client('http://gisdata.usgs.net/XMLWebServices/TNM_Elevation_service.asmx?WSDL')

SAXParseException: <unknown>:7:11: not well-formed (invalid token)

In [3]:
import requests

In [7]:
def elevationQuery(x,y, units='feet'):
    return requests.get('http://nationalmap.gov/epqs/pqs.php?x={x}&y={y}&units={units}&output=json'.format(
        x=x, y=y, units=units)).json()

def elevation(x, y, units='feet'):
    return elevationQuery(x, y, units)['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']
e = elevation(-71.18468999862671, 44.116337862038044)
print e

600.044271


In [6]:
e.json()

{u'USGS_Elevation_Point_Query_Service': {u'Elevation_Query': {u'Data_Source': u'3DEP 1/3 arc-second',
   u'Elevation': 600.044271,
   u'Units': u'Feet',
   u'x': -71.1846899986,
   u'y': 44.116337862}}}

In [10]:
import json
import copy

In [34]:
def elevation_if_zero(coordinates):
    if coordinates[2] == 0:
        time.sleep(.1)
        elev = elevation(coordinates[0], coordinates[1])
        return [coordinates[0], coordinates[1], elev]
    return coordinates
        
elevation_if_zero([
          -71.18685722351074,
          44.14125799208061,
          0
        ])

[-71.18685722351074, 44.14125799208061, 764.484956]

In [26]:
import time

In [38]:
with open('map.geojson', 'r') as geojson_f:
    geojson = json.load(geojson_f)
    new_geojson = copy.deepcopy(geojson)
    new_geojson['features'] = []
    for i, feature in enumerate(geojson['features']):
        print('working on', feature['properties']['name'])
        new_feature = copy.deepcopy(geojson['features'][i])
        
        # clear it once to make sure
        new_feature['geometry']['coordinates'] = []
        
        new_coordinates = []
        
        
        # test if point or line
        try:
            if type(feature['geometry']['coordinates'][0]) == float:
                print 'Point',
                new_coordinates = elevation_if_zero(feature['geometry']['coordinates'])
                print new_coordinates
                
            else:
                print 'List',
                for coord in feature['geometry']['coordinates']:
                    print coord,
                    new_coordinates.append(elevation_if_zero(coord))
                print('Finished list')
        
            new_feature['geometry']['coordinates'] = new_coordinates
        except ValueError:
            new_feature['geometry']['coordinates'] = feature['geometry']['coordinates']
        new_geojson['features'].append(new_feature)
        
        
    with open('elevation_map.geojson', 'w') as elevation_geojson_f:
        json.dump(new_geojson, elevation_geojson_f)
    print new_geojson

 ('working on', u'Inferno Start')
Point [-71.18468999862671, 44.116337862038044, 600.044271]
('working on', u'Run Kayak Transition')
Point [-71.24742150306702, 44.083519827465274, 615.523631]
('working on', u'Kayak - Bike Transition')
Point [-71.17701888084412, 44.10413964702709, 520.290465]
('working on', u'Bike - Hike Transition')
Point [-71.25357449054718, 44.254943692565604, 2027.730266]
('working on', u'Hike - Ski Transition')
Point [-71.29607677459717, 44.2617056110662, 4408.244536]
('working on', u'Top of Ski Course')
Point [-71.30053997039795, 44.25897020133368, 5359.483854]
('working on', u'Glen Ledge Road Interseciton')
Point [-71.18908882141113, 44.12844499859848, 728.981088]
('working on', u'Glen Ledge - Jericho Road Interseciton')
Point [-71.2130355834961, 44.10971331308216, 646.573415]
('working on', u'Jericho Rd Interseciton')
Point [-71.19638442993164, 44.104166611440434, 559.668754]
('working on', u'Glen Road Crossing')
Point [-71.1862564086914, 44.11002144989749, 549.

In [17]:
enumerate?